In [1]:
import pandas as pd
from scipy.stats import f

# Read file
df = pd.read_csv(
    "BulkenDognVerdier.csv",
    sep=";", 
    skiprows=1,            
    decimal=",",           
)

# Convert date and extract year
df["Tidspunkt"] = pd.to_datetime(df["Tidspunkt"], errors="coerce", utc=True)
df["År"] = df["Tidspunkt"].dt.year

# Split into two periods
period1 = df[(df["År"] >= 1961) & (df["År"] <= 1990)]["Vannføring"].dropna()
period2 = df[(df["År"] >= 1991) & (df["År"] <= 2020)]["Vannføring"].dropna()

# Calculate sample variances
s1_sq = period1.var(ddof=1)
s2_sq = period2.var(ddof=1)

# Compute F statistic and degrees of freedom
F = s1_sq / s2_sq
df1 = len(period1) - 1
df2 = len(period2) - 1

# Two-tailed p-value
from scipy.stats import f
p_value = 2 * min(f.cdf(F, df1, df2), 1 - f.cdf(F, df1, df2))

# Print results
print(f"F = {F:.3f}")
print(f"df1 = {df1}, df2 = {df2}")
print(f"p-value = {p_value:.40f}")

# Interpretation
if p_value < 0.05:
    print("Variances are significantly different (reject H0).")
else:
    print("Variances are not significantly different (fail to reject H0).")


F = 0.829
df1 = 10956, df2 = 10957
p-value = 0.0000000000000000000000891753438080690784
Variances are significantly different (reject H0).


In [2]:
from scipy.stats import ttest_ind
import numpy as np

# Two-sample t-test (Welch version)
t_stat, p_value = ttest_ind(period1, period2, equal_var=False)

print(f"t = {t_stat:.3f}")
print(f"p-value = {p_value:.5f}")

if p_value < 0.05:
    print("Mean discharge differs significantly between periods (reject H0).")
else:
    print("No significant difference in mean discharge (fail to reject H0).")

# Printing means of both periods
mean1 = np.mean(period1)
mean2 = np.mean(period2)

print(f"Mean (1961–1990): {mean1:.2f} m³/s")
print(f"Mean (1991–2020): {mean2:.2f} m³/s")


t = -2.811
p-value = 0.00494
Mean discharge differs significantly between periods (reject H0).
Mean (1961–1990): 68.84 m³/s
Mean (1991–2020): 71.57 m³/s


In [7]:
# Convert date and extract year / month
df["Tidspunkt"] = pd.to_datetime(df["Tidspunkt"], dayfirst=True, errors="coerce", utc=True)
df["År"] = df["Tidspunkt"].dt.year
df["Month"] = df["Tidspunkt"].dt.month

# Periods
df1 = df[(df["År"] >= 1961) & (df["År"] <= 1990)]
df2 = df[(df["År"] >= 1991) & (df["År"] <= 2020)]

# Seasonal month groups for winter, spring, summer, autumn
seasons = {
    "Winter_DJF": [12,1,2],
    "Spring_MAM": [3,4,5],
    "Summer_JJA": [6,7,8],
    "Autumn_SON": [9,10,11]
}

# Creating results dictionary
results = {}
 
for season, months in seasons.items():
    p1 = df1[df1["Month"].isin(months)]["Vannføring"].dropna()
    p2 = df2[df2["Month"].isin(months)]["Vannføring"].dropna()

    # F-test
    s1 = p1.var(ddof=1)
    s2 = p2.var(ddof=1)
    F = s1/s2
    df1_f = len(p1)-1
    df2_f = len(p2)-1
    pF = 2 * min(f.cdf(F, df1_f, df2_f), 1 - f.cdf(F, df1_f, df2_f))

    # Welch t-test
    t_stat, p_t = ttest_ind(p1, p2, equal_var=False)
    mean1 = p1.mean()
    mean2 = p2.mean()

    results[season] = {
        "F": F, "pF": pF,
        "t": t_stat, "pt": p_t,
        "mean1": mean1, "mean2": mean2,
        "n1": len(p1), "n2": len(p2)
    }

# Print F test and t test for all seasons
for season, r in results.items():
    print("")
    print(season)
    print("")
    print(f"n1 = {r['n1']}, n2 = {r['n2']}")
    print(f"F = {r['F']:.3f}, p = {r['pF']:.5f}")
    print(f"t = {r['t']:.3f}, p = {r['pt']:.5f}")
    print(f"Mean 1961-1990: {r['mean1']:.2f} m³/s")
    print(f"Mean 1991-2020: {r['mean2']:.2f} m³/s")



Winter_DJF

n1 = 2707, n2 = 2708
F = 0.672, p = 0.00000
t = -6.069, p = 0.00000
Mean 1961-1990: 26.83 m³/s
Mean 1991-2020: 33.86 m³/s

Spring_MAM

n1 = 2760, n2 = 2760
F = 0.840, p = 0.00000
t = -4.449, p = 0.00001
Mean 1961-1990: 62.04 m³/s
Mean 1991-2020: 70.74 m³/s

Summer_JJA

n1 = 2760, n2 = 2760
F = 0.814, p = 0.00000
t = -1.002, p = 0.31645
Mean 1961-1990: 106.69 m³/s
Mean 1991-2020: 108.60 m³/s

Autumn_SON

n1 = 2730, n2 = 2730
F = 0.723, p = 0.00000
t = 3.330, p = 0.00087
Mean 1961-1990: 79.11 m³/s
Mean 1991-2020: 72.40 m³/s
